In [ ]:
# Wczytanie zbioru danych z informacją o tym, czy klient dokonał zakupu 
# oraz czy została mu wysłana oferta reklamowa
import pandas as pd

df = pd.read_csv("klienci2.csv", sep=';')
df

In [ ]:
# Wczytanie drugiego zbioru z klientami
import pandas as pd

df2 = pd.read_csv("klienci-finalnie.csv", sep=';')
df2

In [ ]:
# Ustalenie, czy zbiór zawiera braki danych

liczba_obiektow = df2.shape[0]
liczba_klas = df2.shape[1]
braki = df2.isnull().sum().sum()
obiekty_z_brakami = df2.shape[0] - df2.copy().dropna(how='any').shape[0]
poziom_wypelnienia_kolumn = (liczba_klas * liczba_obiektow - braki) / (liczba_klas * liczba_obiektow)

print(f"liczba obiektów: {liczba_obiektow}\nliczba klas: {liczba_klas}")
print(f"liczba braków danych: {braki}\nliczba obiektów z brakami: {obiekty_z_brakami}")
print(f"poziom wypełnienia kolumn: {poziom_wypelnienia_kolumn * 100}%")

In [ ]:
# Nadanie właściwych nazw kolumnom
df3 = df2.set_axis(['.', 'numer', 'wiek', 'plec', 'wyksztalcenie', 'miejsce', 'kwota', 'wojewodztwo'], axis=1, inplace=False)
df3

In [ ]:
# sprawdzenie kategorii
df3.miejsce.unique()

In [ ]:
# W zbiorze danych znajduje się 100895 obiektów i 8 cech
print(df3.shape)

In [ ]:
# Najwięcej klientów pochodzi ze wsi lub z miejscowości do 10 tys mieszkańców
print(df3.groupby('miejsce').size())

In [ ]:
# Oferta została przesłana do 2006 klientów
print(df.groupby('wysylka_oferty').size())

In [ ]:
# Zakupu dokonało 1317 klientów
print(df.groupby('zakup').size())

In [ ]:
# Połączenie obu zbiorów danych ułatwi przeprowadzenie metod klasyfikacji
df4 = pd.concat([df, df3])
df4

In [ ]:
import seaborn as sns
sns.countplot(df4['miejsce'],label="Count")
plt.show()

In [ ]:
import seaborn as sns
sns.countplot(df4['wyksztalcenie'],label="Count")
plt.show()

In [ ]:
import seaborn as sns
sns.countplot(df4['wojewodztwo'],label="Count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df2.drop('wyksztalcenie', axis=1).plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False, figsize=(9,9), 
                                        title='Box Plot for each input variable')
plt.savefig('wyksztalcenie_box')
plt.show()

In [ ]:
import pylab as pl
df4.drop('wyksztalcenie' ,axis=1).hist(bins=30, figsize=(9,9))
pl.suptitle("Histogram for each numeric input variable")
plt.savefig('wyksztalcenie_hist')
plt.show()

In [ ]:
# Przed przystąpieniem do budowania wykresu korelacji upewniam się,
# że wszystkie zmienne mają odpowiedni typ numeryczny
print(df4.dtypes)

In [ ]:
df4['miejsce'] = pd.to_numeric(df4['miejsce'],errors='coerce')
df4['wyksztalcenie'] = pd.to_numeric(df4['wyksztalcenie'],errors='coerce')
df4['wojewodztwo'] = pd.to_numeric(df4['wojewodztwo'],errors='coerce')

In [ ]:
# Wykres korelacji 

import seaborn as sns

f, ax = pl.subplots(figsize=(10, 8))
corr = df4.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

In [ ]:
# Dokonanie podziału zbioru na testowy i treningowy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Dokładność klasyfikatora drzewa decyzyjnego na treningowym zbiorze danych: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Dokładność klasyfikatora drzewa decyzyjnego na testowym zbiorze danych: {:.2f}'
     .format(clf.score(X_test, y_test)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Dokładność klasyfikatora K-najbliższych sąsiadów na treningowym zbiorze danych: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Dokładność klasyfikatora K-najbliższych sąsiadów na testowym zbiorze danych: {:.2f}'
     .format(knn.score(X_test, y_test)))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('Dokładność Naiwnego Klasyfikatora Bayesa na treningowym zbiorze danych: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Dokładność Naiwnego Klasyfikatora Bayesa na testowym zbiorze danych: {:.2f}'
     .format(gnb.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = knn.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [ ]:
# dummies = pd.get_dummies(df4['wojewodztwo']).rename(columns=lambda x: 'wojewodztwo_' + str(x))
# df4 = pd.concat([df4, dummies], axis=1)
# df4 = df4.drop(['wojewodztwo'], inplace=True, axis=1)
dummy = pd.get_dummies(df4['wojewodztwo'])
dummy.head()